In [1]:
import sys
sys.path.insert(0, './../../Models')
# from CIFAR10_densenet import get_densenet
from sphere_points import generate_points

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.nn.functional import normalize, one_hot
from sklearn.model_selection import train_test_split
from torch.utils.data.dataloader import default_collate

import tracemalloc
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
# from deepspeed.profiling.flops_profiler import get_model_profile

In [2]:
# device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
# device = torch.device("cpu")
# print(device)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
model_loss = nn.CrossEntropyLoss()

In [ ]:
def classifier_head(inp_embedding, classifier_weights, labels, num_classes):
    inp_embedding = normalize(inp_embedding, p=2, dim=-1)
    classifier_output = torch.mm(inp_embedding, classifier_weights)
    classifier_output = torch.softmax(classifier_output, dim=-1)
    loss = model_loss(classifier_output, one_hot(labels, num_classes = num_classes).type(torch.float32))
    return torch.argmax(classifier_output, dim=1).tolist(), loss

In [3]:
def plot_losses(losses):
    n_dims = len(losses)
    plt.figure(figsize = (12, n_dims * 5))
    for l, loss in enumerate(losses):
        epochs = len(loss)
        plt.subplot(n_dims, 1, l + 1)
        plt.plot(1 + np.arange(epochs), loss)
        plt.title(f"Layer {l + 1} Loss")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
    plt.tight_layout()
    plt.show()

In [4]:
theta = 1
cos_similarity = nn.CosineSimilarity(dim = 1, eps = 1e-6)
mse_loss = nn.MSELoss()
num_classes = 10

def loss_layer(vec1, vec2):
    return torch.mean(torch.log(2 - (theta * cos_similarity(vec1, vec2))), dim = 0)

initial = None

# Data dimension
# (num_data, num_features) => no dimension for batch size please
class Layer(nn.Linear):
    def __init__(self, in_features, out_features, bias, device, lr, symm_points):
        super().__init__(in_features, out_features, bias, device)
        self.out_features = out_features
        self.bias_flag = bias
        self.lr = lr
        self.num_classes = num_classes
        self.dimension = out_features
        self.non_linear = nn.LeakyReLU(negative_slope = 0.001)
        self.opt = Adam(self.parameters(), lr = self.lr)
        nn.init.kaiming_normal_(self.weight, mode='fan_in')
        global initial
        self.directions = symm_points
    
    def forward(self, x):
        '''
        x_direction = normalize(x, p = 2, dim = 1)
        '''
        x_direction = x
        if self.bias_flag:
            return self.non_linear(torch.mm(x_direction, self.weight.T) + self.bias.unsqueeze(0))
        else:
            return self.non_linear(torch.mm(x_direction, self.weight.T))

    def train(self, x, label):
        y = self.forward(x) # shape: (num_data, out_features)
        '''
        y = normalize(y, p = 2, dim = 1)
        '''
        directions = torch.zeros_like(y)
        for i in range(y.shape[0]):
            directions[i, :] = self.directions[label[i]]
        
        loss = loss_layer(y, directions)
        self.opt.zero_grad()
        loss.backward(retain_graph = True)
        self.opt.step()

        return loss.item(), y
    
    def test(self, x, top_k):
        with torch.no_grad():
            y = self.forward(x)
        max_idx_list =[]
        for dat in range(y.shape[0]):
            max = -np.inf
            max_idx = 0
            cos_sim_list = []

            for i in range(self.num_classes):
                cos_sim = cos_similarity(y[dat, :].unsqueeze(0), self.directions[i].reshape(1, -1).to(device))
                if top_k == 1:
                    if cos_sim > max:
                        max = cos_sim
                        max_idx = i
                else:
                    cos_sim_list.append(cos_sim)
            
            if top_k == 1:
                max_idx_list.append(max_idx)
            
            else:
                top_k_idx = torch.topk(torch.tensor(cos_sim_list), top_k).indices
                max_idx_list.append(top_k_idx)
        
        if top_k == 1:
            return torch.tensor(max_idx_list), y
        return max_idx_list, y

In [5]:
class Net(nn.Module):
    def __init__(self, dims_list, bias, epochs, lr, device, symm_points_list):
        super(Net, self).__init__()
        self.dims_list = dims_list
        self.bias = bias
        self.epochs = epochs
        self.lr = lr
        self.device = device
        self.mem = -np.inf
        self.layers = []
        self.sigmoid = nn.Sigmoid()
        global initial
        for d in range(len(self.dims_list) - 1):
            print(f"Initialization {d + 1} / {len(self.dims_list) - 1}")
            self.layers += [Layer(self.dims_list[d], self.dims_list[d + 1], self.bias, self.device, self.lr, symm_points_list[d])]
            print("Complete\n")

    def train(self, data_loader):
        layer_loss_list = []
        for i in range(len(self.layers)):
            layer_loss_list.append([])
        pbar = tqdm(total = self.epochs[0] * len(data_loader) * len(self.layers), desc = f"Training", position = 0, leave = True)
        for epoch in range(self.epochs[0]):
            loss_agg = [0] * len(self.layers)
            for dat in data_loader:
                x, label = dat
                for i in range(len(self.layers)):
                    tracemalloc.start()
                    loss, x = self.layers[i].train(x, label)
                    _, peak = tracemalloc.get_traced_memory()
                    tracemalloc.stop()
                    self.mem = max(self.mem, peak / 10 ** 6)
                    loss_agg[i] += loss / len(data_loader)
                    pbar.update(1)
            pbar.set_postfix(epoch = epoch + 1, loss = loss_agg)
            print(self.mem)
            for i in range(len(self.layers)):
                layer_loss_list[i].append(loss_agg[i])
        pbar.close()
        return layer_loss_list
    
    def test(self, data_loader, top_k = 1):
        correct = 0
        total = 0
        for dat in data_loader:
            x, label = dat
            num = label
            preds = []
            for i in range(len(self.layers)):
                pred, x = self.layers[i].test(x, top_k = top_k)
                preds.append(pred)
            
            if top_k == 1:
                correct += (preds[-1].to(device) == num).sum().item()
            else:
                for i in range(num.shape[0]):
                    if num[i].tolist() in preds[-1][i]:
                        correct += 1
            total += num.shape[0]
        return correct / total

In [6]:
datas = []
labels = []
data_dir = "./../../Data/"

remove = ""

# load the features and labels from resnet folder
datas.append(np.load(data_dir + "aircraft100_features_resnet50_pretrained_imagenet" + remove + ".npy"))
labels.append(np.load(data_dir + "aircraft100_labels_resnet50_pretrained_imagenet" + remove + ".npy").astype(np.int64))

datas.append(np.load(data_dir + "cifar10_features_resnet50_pretrained_imagenet" + remove + ".npy"))
labels.append(np.load(data_dir + "cifar10_labels_resnet50_pretrained_imagenet" + remove + ".npy").astype(np.int64))

datas.append(np.load(data_dir + "cifar100_features_resnet50_pretrained_imagenet" + remove + ".npy"))
labels.append(np.load(data_dir + "cifar100_labels_resnet50_pretrained_imagenet" + remove + ".npy").astype(np.int64))

datas.append(np.load(data_dir + "flowers102_features_resnet50_pretrained_imagenet" + remove + ".npy"))
labels.append(np.load(data_dir + "flowers102_labels_resnet50_pretrained_imagenet" + remove + ".npy").astype(np.int64))

datas.append(np.load(data_dir + "food101_features_resnet50_pretrained_imagenet" + remove + ".npy"))
labels.append(np.load(data_dir + "food101_labels_resnet50_pretrained_imagenet" + remove + ".npy").astype(np.int64))

datas.append(np.load(data_dir + "pets37_features_resnet50_pretrained_imagenet" + remove + ".npy"))
labels.append(np.load(data_dir + "pets37_labels_resnet50_pretrained_imagenet" + remove + ".npy").astype(np.int64))

datas_name = ['aircraft100', 'cifar10', 'cifar100', 'flowers102', 'food101', 'pets37']
#  datas_name = ['food101']
# datas_name = ['aircraft100']

In [7]:
def generate_symm_points(layer_dims, num_classes):
    vecs = []
    for k in layer_dims:
        directions = generate_points(num_classes, k, steps = 10000, initial_points = None)
        directions = [torch.tensor(t, dtype = torch.float32) for t in directions]
        vecs.append(directions)
    return vecs

In [8]:
num_runs = 1

for run_id in [2, 3, 4]:
    for i in range(len(datas_name)):
        data = datas[i]
        label = labels[i]
        print(f"Dataset: {datas_name[i]}")
        ratios = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
        # ratios = [0.99, 0.1]
    
        initial = None
        accuracy_train_list_1, accuracy_train_list_k = [], []
        accuracy_test_list_1, accuracy_test_list_k = [], []
    
        num_classes = len(np.unique(label))
        mem_list = []
        for r in ratios:
            print("Test to train ratio: ", r)
            test_size = r
            batch_size = 32    
    
            train_data, test_data, train_labels, test_labels = train_test_split(
                                                                                    data, 
                                                                                    label, 
                                                                                    test_size = test_size, 
                                                                                    random_state = 100
                                                                                )
    
            train_loader = DataLoader(list(zip(train_data, train_labels)), 
                                        batch_size = batch_size, shuffle = True, 
                                        collate_fn = lambda x: tuple(x_.to(device) for x_ in default_collate(x)))
    
            test_loader = DataLoader(list(zip(test_data, test_labels)), 
                                        batch_size = batch_size, shuffle = True, 
                                        collate_fn = lambda x: tuple(x_.to(device) for x_ in default_collate(x)))
    
            print(f"Train Loader: {len(train_loader)}")
            print(f"Test Loader: {len(test_loader)}")
            
            print(f"Network Size: {data.shape[1]}")
            dims_list = [data.shape[1], 1024, 1024]
            if num_classes < 50:
                dims_list = [data.shape[1], 512, 64]
            bias = True
            epochs = [50, 50]
            lr = 1.0e-3
            symmetric_points_list = generate_symm_points(dims_list[1 : ], num_classes)
    
            net = Net(dims_list, bias, epochs, lr, device, symmetric_points_list).to(device)
            
            layer_losses = net.train(train_loader)
            mem_list.append(net.mem)
    
            accuracy_train_1 = net.test(train_loader, top_k = 1)
            print(f"Training Accuracy: {accuracy_train_1 * 100}%")
            accuracy_train_list_1.append(accuracy_train_1)
            
            accuracy_test_1 = net.test(test_loader, top_k = 1)
            print(f"Testing Accuracy: {accuracy_test_1 * 100}%")
            accuracy_test_list_1.append(accuracy_test_1)
    
            accuracy_train_k = net.test(train_loader, top_k = 5)
            print(f"Top 5 Training Accuracy: {accuracy_train_k * 100}%")
            accuracy_train_list_k.append(accuracy_train_k)
            
            accuracy_test_k = net.test(test_loader, top_k = 5)
            print(f"Top 5 Testing Accuracy: {accuracy_test_k * 100}%")
            accuracy_test_list_k.append(accuracy_test_k)
            # plot_losses(layer_losses)
            
        # write the losses and accuracies to a file
        file_name = f"./../Results/TL/New/{datas_name[i] + '_' + str(run_id)}" + remove  + "_Openloop.txt"
        with open(file_name, 'w') as f:
            # write the training and testing accuracies
            f.write("Top 1: Training Accuracies\n")
            for acc in accuracy_train_list_1:
                f.write(f"{acc}\n")
            f.write("Top 1: Testing Accuracies\n")
            for acc in accuracy_test_list_1:
                f.write(f"{acc}\n")
            
            f.write("\nTop 5: Training Accuracies\n")
            for acc in accuracy_train_list_k:
                f.write(f"{acc}\n")
            f.write("Top 5: Testing Accuracies\n")
            for acc in accuracy_test_list_k:
                f.write(f"{acc}\n")
            
            # write the losses
            f.write("\nLayer Losses\n")
            for losses in layer_losses:
                for loss in losses:
                    f.write(f"{loss}\n")
                f.write("\n")
    
            # write the model summary
            f.write("Model Summary\n")
            f.write(str(net))
            f.write("\n")
            
            # write the model parameters
            f.write("Model Parameters\n")
            f.write(f"Dimensions List: {dims_list}\n")
            f.write(f"Bias: {bias}\n")
            f.write(f"Epochs: {epochs}\n")
            f.write(f"Learning Rate: {lr}\n")
            f.write(f"Device: {device}\n")
            f.write("\n")

            #     # write the memory usage
        #     f.write("\nMemory Usage\n")
        #     for i, mem in enumerate(mem_list):
        #         f.write(f"Ratio: {ratios[i]}\n")
        #         f.write(f"{mem} MB\n")

Dataset: food101
Test to train ratio:  0.1
Train Loader: 188
Test Loader: 21
Network Size: 2048
Initialization 1 / 2
Complete

Initialization 2 / 2
Complete



Training:   2%|██                                                                                              | 393/18800 [00:03<02:33, 120.26it/s, epoch=1, loss=[0.6833077123824589, 0.6901023321329278]]

0.028797


Training:   4%|███▉                                                                                            | 772/18800 [00:06<02:32, 118.24it/s, epoch=2, loss=[0.6619427980894739, 0.6467623082881278]]

0.028797


...

0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409


Training:  56%|██████████████████████████████████████████████████████████▏                                             | 168/300 [00:00<00:00, 812.89it/s, epoch=28, loss=[0.4758197863896687, 0.5275979240735372]]

0.005409


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 865.87it/s, epoch=50, loss=[0.4266233046849569, 0.38540170590082806]]


0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
0.005409
Training Accuracy: 100.0%
Testing Accuracy: 5.211331616421754%
Top 5 Training Accuracy: 100.0%
Top 5 Testing Accuracy: 14.588698682017876%
